In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf

import gzip
import os
import re
import sys
import tarfile

from six.moves import urllib
import tensorflow as tf

from tensorflow.models.image.cifar10 import cifar10_input
import numpy as np
from data_utils import *
from batch_norm import *

BATCH_SIZE=256
DATA_DIR='/tmp/cifar10_data'
IMAGE_SIZE = cifar10_input.IMAGE_SIZE
NUM_CLASSES = cifar10_input.NUM_CLASSES
NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN = cifar10_input.NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN
NUM_EXAMPLES_PER_EPOCH_FOR_EVAL = cifar10_input.NUM_EXAMPLES_PER_EPOCH_FOR_EVAL


# Constants describing the training process.
MOVING_AVERAGE_DECAY = 0.9999     # The decay to use for the moving average.
NUM_EPOCHS_PER_DECAY = 350.0      # Epochs after which learning rate decays.

TOWER_NAME = 'tower'
def _activation_summary(x):
    # Remove 'tower_[0-9]/' from the name in case this is a multi-GPU training
    # session. This helps the clarity of presentation on tensorboard.
    tensor_name = re.sub('%s_[0-9]*/' % TOWER_NAME, '', x.op.name)
    tf.histogram_summary(tensor_name + '/activations', x)
    tf.scalar_summary(tensor_name + '/sparsity', tf.nn.zero_fraction(x))


def _variable_on_cpu(name, shape, initializer):
    with tf.device('/cpu:0'):
        var = tf.get_variable(name, shape, initializer=initializer)
    return var

def _variable_with_weight_decay(name, shape, stddev, wd):
    var = _variable_on_cpu(name, shape,
                         tf.truncated_normal_initializer(stddev=stddev))
    if wd is not None:
        weight_decay = tf.mul(tf.nn.l2_loss(var), wd, name='weight_loss')
        tf.add_to_collection('losses', weight_decay)
    return var



# %% Setup input to the network and true output label.  These are
# simply placeholders which we'll fill in later.
x = tf.placeholder(tf.float32, [None, 32,32,3])
y = tf.placeholder(tf.float32, [None, 20])

# %% We add a new type of placeholder to denote when we are training.
# This will be used to change the way we compute the network during
# training/testing.
is_training = tf.placeholder(tf.bool, name='is_training')

# %% We'll convert our MNIST vector data to a 4-D tensor:
# N x W x H x C
initfact=10
lr=.1
path='dataset'


kernel = _variable_with_weight_decay('conv0',
                                 shape=[3, 3, 3, 16],
                                 stddev=np.sqrt(2.0/initfact/3)
                                 , wd=0.0)
net = tf.nn.conv2d(x, kernel, [1, 1, 1, 1], padding='SAME')
#net = BatchNorm(net)
net = batch_norm(net,is_training,scope='conv0')
net = tf.nn.elu(net)
#_activation_summary(net)
    

for block in range(3):
    nfilters=16<<block
    for layer in range(1):
        net_copy=net
        for i in range(2):
            name = 'block_%d/layer_%d/conv%d' % (block, layer,i)
            if block==0:
                i=1
            kernel = _variable_with_weight_decay(name,
                    shape=[3, 3,
                           net.get_shape().as_list()[3],
                           nfilters],
                    stddev=np.sqrt(2.0/initfact/nfilters),
                    wd=0.0)
            if layer==0 and block!=0 and i==0:
                up=1
            else:
                up=0
            net  = tf.nn.conv2d(net,
                    kernel,
                    [1,1+up,1+up, 1],
                    padding='SAME')
            net = batch_norm(net,scope=name)
            #net = BatchNorm(net)
            net = tf.nn.elu(net)
            _activation_summary(net)


# residual function (identity shortcut)
        if net_copy.get_shape().as_list()[1]!=net.get_shape().as_list()[1]:
            net_copy=tf.nn.avg_pool(net_copy,[1,2,2,1],
                    strides=[1,2,2,1],padding='VALID')
            net_copy=tf.pad(net_copy,[[0,0],[0,0],[0,0],[0,int(nfilters/2)]])
        net = net + net_copy

#Global avg pooling
net_shape = net.get_shape().as_list()
net = tf.nn.avg_pool(net,
              ksize=[1, net_shape[1], net_shape[2], 1],
              strides=[1, 1, 1, 1], 
              padding='VALID',name='global_pooling')
net_shape = net.get_shape().as_list()
net = tf.reshape(net,
        [-1, net_shape[1] * net_shape[2] * net_shape[3]])

weights = _variable_with_weight_decay('softmax_w',
    [64, NUM_CLASSES],
    stddev=1/64.0,
    wd=0.0)
biases = _variable_on_cpu('softmax_b',
    [NUM_CLASSES],
    tf.constant_initializer(0.0))
softmax_linear = tf.add(tf.matmul(net, weights), biases, name='softmax')
y = tf.cast(y, tf.int64)
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
      softmax_linear, y, name='cross_entropy_per_example')
cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy')
 
train_step=tf.train.MomentumOptimizer(lr,.9).minimize(cross_entropy_mean)
#_activation_summary(softmax_linear)

correct_prediction=tf.equal(tf.argmax(softmax_linear,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,'float')

# %% We now create a new session to actually perform the initialization the
# variables:
sess=tf.Session()
sess.run(tf.initialize_all_variables())

# %% We'll train in minibatches and report accuracy:
n_epochs = 10
batch_size = 100
# load data
Xtr, Ytr, Xte, Yte=loadCIFAR100(path)
numTrain=len(Xtr)
for epoch_i in range(n_epochs):
    for batch_i in range(mnist.train.num_examples // batch_size):
        idx=np.random.choice(numTrain,batch_size)
        batch_xs=Xtr(idx)
        batch_ys=Ytr(idx)
        sess.run(train_step, feed_dict={
            x: batch_xs, y: batch_ys, is_training: True})
    print(sess.run(accuracy,
                   feed_dict={
                       x: Xte,
                       y: Yte,
                       is_training: False
                   }))


SyntaxError: invalid syntax (<ipython-input-1-5987ad60c7b8>, line 151)